# GasHolders models

In [2]:
# Import useful libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import sklearn
import datetime
import yfinance as yf
import seaborn as sns
import random
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.signal import StateSpace
from typing import Optional, List
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import pandas as pd
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


In [3]:
# Useful functions

def information(input):
    print("The array is : \n", input)
    print(" ")
    print("type:", type(input))
    print("sys.getsizeof (Python object size) KB:", sys.getsizeof(input) / (1024))
    print("dtype:", input.dtype)
    print("shape:", input.shape)
    print(" ")
        

## State Space model BFGH

Using standard class definition as suggested


def __init__ ( self, **params):

"""

Initialize the digital twin with optional model paramters


Paramters (optional):

param1(type) : Description of the parameter

param1(type) : Description of the parameter

"""

self.params = params    

def __call__(self, inputs: dict) -> dict :

"""

Execute the digital twin simulation based on input variables

Expected inputs:

    - input1 (type) : Description

    - input2 (type) : Description

Returns : 

    dict with:

        - output1 (type) : Description

        - output2 (type) : Description


"""

Extract inputs with deafult values

input1 = inputs.get("input1", 0)

input2 = inputs.get("input2", 0)

output1 = f(input1)

output2 = g(input2)

return {

    "output1": output1,

    "output2": output2 
    
}


We describe the system using a set of first-order differential equations:

$$
\begin{aligned}
\dot{x}(t) &= A x(t) + B u(t) \\
y(t) &= C x(t) + D u(t)
\end{aligned}
$$

or 

$$
\begin{aligned}
x_{k+1} &= A x_k + B u_k \\
y_k &= C x_k + D u_k
\end{aligned}
$$


Where:

- $ x \in \mathbb{R}^n $ is the **state vector**
- $ u \in \mathbb{R}^m $ is the **input vector**
- $ y \in \mathbb{R}^p $ is the **output vector**
- $ A \in \mathbb{R}^{n \times n} $ is the **system matrix**
- $ B \in \mathbb{R}^{n \times m} $ is the **input matrix**
- $ C \in \mathbb{R}^{p \times n} $ is the **output matrix**
- $ D \in \mathbb{R}^{p \times m} $ is the **feedthrough matrix**

In our model 

- $ x \in \mathbb{R}^1 $ is the **state vector**
- $ u \in \mathbb{R}^1 $ is the **input vector**
- $ y \in \mathbb{R}^1 $ is the **output vector**
- $ A \in \mathbb{R}^{1 \times 1} $ is the **system matrix**
- $ B \in \mathbb{R}^{1 \times 1} $ is the **input matrix**
- $ C \in \mathbb{R}^{1 \times 1} $ is the **output matrix**
- $ D \in \mathbb{R}^{1 \times 1} $ is the **feedthrough matrix**

In particular : 
$$
u_k = \begin{bmatrix}
u^1_k \\
\end{bmatrix}
$$


- $ u^i_k \in \mathbb{R} $ = output gas at step $k$ $[kNm^3/h]$


and $$ u^{i}_k = {\dot{v}_k}_{\text{BFG}}  \; \; \; \; [kNm^3/h]$$

and $$ y_k = \text{Percentage of Gasholder level with respect to its maximum  } \; \; \; \; [ ]$$ 


In [4]:
# Define BFGH state space

# Define A,B,C,D
A_BFGH = np.array([0.994])

B_BFGH = np.array( [3.135*1e-6]) # possibility of multply this by 1000

C_BFGH = np.array( [3.49416 * 1e6])


# Initial state
x_BFGH = np.array([0.004])

In [5]:
print("A is :")
information(A_BFGH)
print("\n")

print("B is :")
information(B_BFGH)
print("\n")

print("C is :")
information(C_BFGH)
print("\n")


A is :
The array is : 
 [0.994]
 
type: <class 'numpy.ndarray'>
sys.getsizeof (Python object size) KB: 0.1171875
dtype: float64
shape: (1,)
 


B is :
The array is : 
 [3.135e-06]
 
type: <class 'numpy.ndarray'>
sys.getsizeof (Python object size) KB: 0.1171875
dtype: float64
shape: (1,)
 


C is :
The array is : 
 [3494160.]
 
type: <class 'numpy.ndarray'>
sys.getsizeof (Python object size) KB: 0.1171875
dtype: float64
shape: (1,)
 




In [16]:
class BFGH :
    """
    Class describing BFG Gasholder
    
    Parameters :
    
    State dimension : n = 1, 
    Input dimension : m = 1, 
    Output dimensions : p = 1
    
    A : n x n (ndarray, dtype = float)
    B : n x m (ndarray, dtype = float)
    C : p x n (ndarray, dtype = float)
    D : p x m (ndarray, dtype = float)
    x0: n x 1 (ndarray, dtype = float)
    
    Input : 
    u : m dimensional vector containing output gas [kNm^3/h]
    
    Output :
    y : p dimensional vector containing GasHolder Level []

    """
    
    def __init__(self, A : np.ndarray = A_BFGH  , B : np.ndarray = B_BFGH, C : np.ndarray = C_BFGH, D : np.ndarray = 0, x0 : np.ndarray = x_BFGH, 
                 input_names : Optional[List[str]] = None, output_names : Optional[List[str]] = None) :
        """
        
        Parameters : 
        
        A : n x n (ndarray, dtype = float)
        B : n x m (ndarray, dtype = float)
        C : p x n (ndarray, dtype = float)
        D : p x m (ndarray, dtype = float)
        x0: n x 1 (ndarray, dtype = float)
        input_names : List of strings ( m dimensional ) ; type([List[str]])
        output_names : List of strings ( p dimensional ) ; type([List[str]])
        
        """
        
        # Declare state space
        self.A = np.array(A, dtype = float)
        self.B = np.array(B, dtype = float)
        self.C = np.array(C, dtype = float)
        self.D = np.array(D, dtype = float)
        self.x = np.array(x0, dtype = float)
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # Dimnension check
        assert self.A.shape == (n,n), f" A must be a square matrix {n} x {n}"
        assert self.B.shape == (n,m), f" B must be a matrix {n} x {m}"
        assert self.C.shape == (p,n), f" C must be a matrix {p} x {n}"
        assert self.D.shape == (p,m), f" D must be a matrix {p} x {m}"
        
        self.input_names = input_names if input_names else [f"u{i}" for i in range(m)]
        self.output_names = output_names if output_names else [f"y{i}" for i in range(p)]
        
    
    def __call__(self, input_dict : dict) -> dict:
        """
        
        Compute output y_{k} = C * x_{k} + D * u_{k} given current state and input.
        
        Compute  x_{k+1} = A * x_{k} + B * u_{k} given current state and input.
        
        input_dict: {input_name: value} --> {input_name_1 : value_1, input_name_2 : value_2, ..., }
        
        return: {output_name: value} --> {output_name_1 : value_1, output_name_2 : value_2, ...., }
        
        """
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # create u vector of m dimension with values in input names
        u = np.array ( [input_dict[name] for name in self.input_names ], dtype = float )
        
        assert u.shape == (m,), f"u must be {m} dimensional"
        
        # update x_{k+1} = A * x_{k} + B * u_{k}
        self.x = self.A @ self.x + self.B @ u
        
        assert self.x.shape == (n,), f"x must be {n} dimensional"
        
        y = self.C @ self.x + self.D @ u
        
        assert self.y.shape == (p,), f"y must be {p} dimensional"
        
        return {name: y[i] for i, name in enumerate(self.output_names) }

## State Space model BOFGH

Using standard class definition as suggested


def __init__ ( self, **params):

"""

Initialize the digital twin with optional model paramters


Paramters (optional):

param1(type) : Description of the parameter

param1(type) : Description of the parameter

"""

self.params = params    

def __call__(self, inputs: dict) -> dict :

"""

Execute the digital twin simulation based on input variables

Expected inputs:

    - input1 (type) : Description

    - input2 (type) : Description

Returns : 

    dict with:

        - output1 (type) : Description

        - output2 (type) : Description


"""

Extract inputs with deafult values

input1 = inputs.get("input1", 0)

input2 = inputs.get("input2", 0)

output1 = f(input1)

output2 = g(input2)

return {

    "output1": output1,

    "output2": output2 
    
}


We describe the system using a set of first-order differential equations:

$$
\begin{aligned}
\dot{x}(t) &= A x(t) + B u(t) \\
y(t) &= C x(t) + D u(t)
\end{aligned}
$$

or 

$$
\begin{aligned}
x_{k+1} &= A x_k + B u_k \\
y_k &= C x_k + D u_k
\end{aligned}
$$


Where:

- $ x \in \mathbb{R}^n $ is the **state vector**
- $ u \in \mathbb{R}^m $ is the **input vector**
- $ y \in \mathbb{R}^p $ is the **output vector**
- $ A \in \mathbb{R}^{n \times n} $ is the **system matrix**
- $ B \in \mathbb{R}^{n \times m} $ is the **input matrix**
- $ C \in \mathbb{R}^{p \times n} $ is the **output matrix**
- $ D \in \mathbb{R}^{p \times m} $ is the **feedthrough matrix**

In our model 

- $ x \in \mathbb{R}^1 $ is the **state vector**
- $ u \in \mathbb{R}^1 $ is the **input vector**
- $ y \in \mathbb{R}^1 $ is the **output vector**
- $ A \in \mathbb{R}^{1 \times 1} $ is the **system matrix**
- $ B \in \mathbb{R}^{1 \times 1} $ is the **input matrix**
- $ C \in \mathbb{R}^{1 \times 1} $ is the **output matrix**
- $ D \in \mathbb{R}^{1 \times 1} $ is the **feedthrough matrix**

In particular : 
$$
u_k = \begin{bmatrix}
u^1_k \\
\end{bmatrix}
$$


- $ u^i_k \in \mathbb{R} $ = output gas at step $k$ $[kNm^3/h]$


and $$ u^{i}_k = {\dot{v}_k}_{\text{BOFG}}  \; \; \; \; [kNm^3/h]$$

and $$ y_k = \text{Percentage of Gasholder level with respect to its maximum  } \; \; \; \; [ ]$$ 


In [11]:
# Define BOFGH state space

# Define A,B,C,D
A_BOFGH = np.array([0.999])

B_BOFGH = np.array( [7.2884*1e-6]) # possibility of multply this by 1000

C_BOFGH = np.array( [3.2121 * 1e6])


# Initial state
x_BOFGH = np.array([0.0049])

In [17]:
class BOFGH :
    """
    Class describing BOFG Gasholder
    
    Parameters :
    
    State dimension : n = 1, 
    Input dimension : m = 1, 
    Output dimensions : p = 1
    
    A : n x n (ndarray, dtype = float)
    B : n x m (ndarray, dtype = float)
    C : p x n (ndarray, dtype = float)
    D : p x m (ndarray, dtype = float)
    x0: n x 1 (ndarray, dtype = float)
    
    Input : 
    u : m dimensional vector containing output gas [kNm^3/h]
    
    Output :
    y : p dimensional vector containing GasHolder Level []

    """
    
    def __init__(self, A : np.ndarray = A_BOFGH  , B : np.ndarray = B_BOFGH, C : np.ndarray = C_BOFGH, D : np.ndarray = 0, x0 : np.ndarray = x_BOFGH, 
                 input_names : Optional[List[str]] = None, output_names : Optional[List[str]] = None) :
        """
        
        Parameters : 
        
        A : n x n (ndarray, dtype = float)
        B : n x m (ndarray, dtype = float)
        C : p x n (ndarray, dtype = float)
        D : p x m (ndarray, dtype = float)
        x0: n x 1 (ndarray, dtype = float)
        input_names : List of strings ( m dimensional ) ; type([List[str]])
        output_names : List of strings ( p dimensional ) ; type([List[str]])
        
        """
        
        # Declare state space
        self.A = np.array(A, dtype = float)
        self.B = np.array(B, dtype = float)
        self.C = np.array(C, dtype = float)
        self.D = np.array(D, dtype = float)
        self.x = np.array(x0, dtype = float)
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # Check
        assert self.A.shape == (n,n), f" A must be a square matrix {n} x {n}"
        assert self.B.shape == (n,m), f" B must be a matrix {n} x {m}"
        assert self.C.shape == (p,n), f" C must be a matrix {p} x {n}"
        assert self.D.shape == (p,m), f" D must be a matrix {p} x {m}"
        
        self.input_names = input_names if input_names else [f"u{i}" for i in range(m)]
        self.output_names = output_names if output_names else [f"y{i}" for i in range(p)]
        
    
    def __call__(self, input_dict : dict) -> dict:
        """
        
        Compute output y_{k} = C * x_{k} + D * u_{k} given current state and input.
        Compute  x_{k+1} = A * x_{k} + B * u_{k} given current state and input.
        
        input_dict: {input_name: value} --> {input_name_1 : value_1, input_name_2 : value_2, ..., }
        return: {output_name: value} --> {output_name_1 : value_1, output_name_2 : value_2, ...., }
        
        """
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # create u vector of m dimension with values in input names
        u = np.array ( [input_dict[name] for name in self.input_names ], dtype = float )
        
        assert u.shape == (m,), f"u must be {m} dimensional"
        
        # update x_{k+1} = A * x_{k} + B * u_{k}
        self.x = self.A @ self.x + self.B @ u
        
        assert self.x.shape == (n,), f"x must be {n} dimensional"
        
        y = self.C @ self.x + self.D @ u
        
        assert self.y.shape == (p,), f"y must be {p} dimensional"
        
        return {name: y[i] for i, name in enumerate(self.output_names) }

## State Space model COGH

Using standard class definition as suggested


def __init__ ( self, **params):

"""

Initialize the digital twin with optional model paramters


Paramters (optional):

param1(type) : Description of the parameter

param1(type) : Description of the parameter

"""

self.params = params    

def __call__(self, inputs: dict) -> dict :

"""

Execute the digital twin simulation based on input variables

Expected inputs:

    - input1 (type) : Description

    - input2 (type) : Description

Returns : 

    dict with:

        - output1 (type) : Description

        - output2 (type) : Description


"""

Extract inputs with deafult values

input1 = inputs.get("input1", 0)

input2 = inputs.get("input2", 0)

output1 = f(input1)

output2 = g(input2)

return {

    "output1": output1,

    "output2": output2 
    
}


We describe the system using a set of first-order differential equations:

$$
\begin{aligned}
\dot{x}(t) &= A x(t) + B u(t) \\
y(t) &= C x(t) + D u(t)
\end{aligned}
$$

or 

$$
\begin{aligned}
x_{k+1} &= A x_k + B u_k \\
y_k &= C x_k + D u_k
\end{aligned}
$$


Where:

- $ x \in \mathbb{R}^n $ is the **state vector**
- $ u \in \mathbb{R}^m $ is the **input vector**
- $ y \in \mathbb{R}^p $ is the **output vector**
- $ A \in \mathbb{R}^{n \times n} $ is the **system matrix**
- $ B \in \mathbb{R}^{n \times m} $ is the **input matrix**
- $ C \in \mathbb{R}^{p \times n} $ is the **output matrix**
- $ D \in \mathbb{R}^{p \times m} $ is the **feedthrough matrix**

In our model 

- $ x \in \mathbb{R}^1 $ is the **state vector**
- $ u \in \mathbb{R}^1 $ is the **input vector**
- $ y \in \mathbb{R}^1 $ is the **output vector**
- $ A \in \mathbb{R}^{1 \times 1} $ is the **system matrix**
- $ B \in \mathbb{R}^{1 \times 1} $ is the **input matrix**
- $ C \in \mathbb{R}^{1 \times 1} $ is the **output matrix**
- $ D \in \mathbb{R}^{1 \times 1} $ is the **feedthrough matrix**

In particular : 
$$
u_k = \begin{bmatrix}
u^1_k \\
\end{bmatrix}
$$


- $ u^i_k \in \mathbb{R} $ = output gas at step $k$ $[kNm^3/h]$


and $$ u^{i}_k = {\dot{v}_k}_{\text{COG}}  \; \; \; \; [kNm^3/h]$$

and $$ y_k = \text{Percentage of Gasholder level with respect to its maximum  } \; \; \; \; [ ]$$ 


In [13]:
# Define COGH state space

# Define A,B,C,D
A_COGH = np.array([1])

B_COGH = np.array( [1.7187*1e-6]) # possibility of multply this by 1000

C_COGH = np.array( [1.0066 * 1e6])


# Initial state
x_COGH = np.array([0.0045])

In [15]:
class COGH :
    """
    Class describing COG Gasholder
    
    Parameters :
    
    State dimension : n = 1, 
    Input dimension : m = 1, 
    Output dimensions : p = 1
    
    A : n x n (ndarray, dtype = float)
    B : n x m (ndarray, dtype = float)
    C : p x n (ndarray, dtype = float)
    D : p x m (ndarray, dtype = float)
    x0: n x 1 (ndarray, dtype = float)
    
    Input : 
    u : m dimensional vector containing output gas [kNm^3/h]
    
    Output :
    y : p dimensional vector containing GasHolder Level []

    """
    
    def __init__(self, A : np.ndarray = A_COGH  , B : np.ndarray = B_COGH, C : np.ndarray = C_COGH, D : np.ndarray = 0, 
                 x0 : np.ndarray = x_COGH, input_names : Optional[List[str]] = None, output_names : Optional[List[str]] = None) :
        """
        
        Parameters : 
        
        A : n x n (ndarray, dtype = float)
        B : n x m (ndarray, dtype = float)
        C : p x n (ndarray, dtype = float)
        D : p x m (ndarray, dtype = float)
        x0: n x 1 (ndarray, dtype = float)
        input_names : List of strings ( m dimensional ) ; type([List[str]])
        output_names : List of strings ( p dimensional ) ; type([List[str]])
        
        """
        
        # Declare state space
        self.A = np.array(A, dtype = float)
        self.B = np.array(B, dtype = float)
        self.C = np.array(C, dtype = float)
        self.D = np.array(D, dtype = float)
        self.x = np.array(x0, dtype = float)
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # Check
        assert self.A.shape == (n,n), f" A must be a square matrix {n} x {n}"
        assert self.B.shape == (n,m), f" B must be a matrix {n} x {m}"
        assert self.C.shape == (p,n), f" C must be a matrix {p} x {n}"
        assert self.D.shape == (p,m), f" D must be a matrix {p} x {m}"
        
        self.input_names = input_names if input_names else [f"u{i}" for i in range(m)]
        self.output_names = output_names if output_names else [f"y{i}" for i in range(p)]
        
    
    def __call__(self, input_dict : dict) -> dict:
        """
        
        Compute output y_{k} = C * x_{k} + D * u_{k} given current state and input.
        Compute  x_{k+1} = A * x_{k} + B * u_{k} given current state and input.
        
        input_dict: {input_name: value} --> {input_name_1 : value_1, input_name_2 : value_2, ..., }
        return: {output_name: value} --> {output_name_1 : value_1, output_name_2 : value_2, ...., }
        
        """
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # create u vector of m dimension with values in input names
        # u = [u1, u2,.., u_m]
        u = np.array ( [input_dict[name] for name in self.input_names ], dtype = float )
        
        assert u.shape == (m,), f"u must be {m} dimensional"
        
        # update x_{k+1} = A * x_{k} + B * u_{k}
        self.x = self.A @ self.x + self.B @ u
        
        assert self.x.shape == (n,), f"x must be {n} dimensional"
        
        y = self.C @ self.x + self.D @ u
        
        assert self.y.shape == (p,), f"y must be {p} dimensional"
        
        return {name: y[i] for i, name in enumerate(self.output_names) }